In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import xarray as xr

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import metpy.calc as mpcalc
from metpy.units import units

import hvplot.xarray
#import hvplot

from uviz.utils.tools import find_TC_bbox
#from uviz.datashader_tools.utils import datashader_wrapper
from uviz.plotting.utils import basin_bboxes, ssh_wsp, ssh_mslp, sshws_color, geog_features

In [ ]:
prototype_dir = r"/gpfs/group/cmz5202/default/cnd5285/synth_events"
storm_1279_dir = os.path.join(prototype_dir, "VR28.NATL.EXT.CAM5.4CLM5.0.dtime900_storm_1279")
parent_28km = os.path.join(storm_1279_dir, '28km')
child_3km = os.path.join(storm_1279_dir, '3km')

In [ ]:
h1p_files = glob.glob(os.path.join(parent_28km, '*.h1.*.nc'))
h2p_files = glob.glob(os.path.join(parent_28km, '*.h2.*.nc'))
h3p_files = glob.glob(os.path.join(parent_28km, '*.h3.*.nc'))
h4p_files = glob.glob(os.path.join(parent_28km, '*.h4.*.nc'))

# CAM mesh is the "ext" one, based on original model run. Other options are "ref" or "wat"
p_mesh = r"/gpfs/group/cmz5202/default/cnd5285/maps_and_grids/ne0np4natlanticext.ne30x4.g_scrip.nc"

h1c_files = glob.glob(os.path.join(child_3km, '*.h1.*.nc'))
h2c_files = glob.glob(os.path.join(child_3km, '*.h2.*.nc'))
h3c_files = glob.glob(os.path.join(child_3km, '*.h3.*.nc'))
h4c_files = glob.glob(os.path.join(child_3km, '*.h4.*.nc'))
h5c_files = os.listdir(os.path.join(child_3km, 'h5'))

# Dug in the attributes for the mesh, TODO: make function that finds it automatically
c_mesh = "/gpfs/group/cmz5202/default/cnd5285/MPAS_3km/x20.835586.florida.init.CAM.nc"

In [ ]:
# Native grid files
parallel = True
h1pn_ds = xr.open_mfdataset([f for f in h1p_files if 'remap' not in f], parallel=parallel)
h2pn_ds = xr.open_mfdataset([f for f in h2p_files if 'remap' not in f], parallel=parallel)
h3pn_ds = xr.open_mfdataset([f for f in h3p_files if 'remap' not in f], parallel=parallel)
h4pn_ds = xr.open_mfdataset([f for f in h4p_files if 'remap' not in f], parallel=parallel)
p_mesh_ds = xr.open_dataset(p_mesh)

# Regridded files
h1pr_ds = xr.open_mfdataset([f for f in h1p_files if 'remap' in f], parallel=parallel)
h2pr_ds = xr.open_mfdataset([f for f in h2p_files if 'remap' in f], parallel=parallel)
h3pr_ds = xr.open_mfdataset([f for f in h3p_files if 'remap' in f], parallel=parallel)
h4pr_ds = xr.open_mfdataset([f for f in h4p_files if 'remap' in f], parallel=parallel)

# Native grid files
h1cn_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' not in f], parallel=parallel)
h2cn_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' not in f], parallel=parallel)
h3cn_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' not in f], parallel=parallel)
h4cn_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' not in f], parallel=parallel)
#h5cn_ds = xr.open_mfdataset([f for f in h5c_files])
c_mesh_ds = xr.open_dataset(c_mesh, chunks={'nCells':10000, 'nVertices':100, 'nEdges':100})

# Regridded files
h1cr_ds = xr.open_mfdataset([f for f in h1c_files if 'remap' in f], parallel=parallel)
h2cr_ds = xr.open_mfdataset([f for f in h2c_files if 'remap' in f], parallel=parallel)
h3cr_ds = xr.open_mfdataset([f for f in h3c_files if 'remap' in f], parallel=parallel)
h4cr_ds = xr.open_mfdataset([f for f in h4c_files if 'remap' in f], parallel=parallel)

In [ ]:
h2pr_ds['VORT'] = mpcalc.vorticity(h2pr_ds['U'], h2pr_ds['V'])

In [ ]:
# Kernel dies trying to calculate MPAS vorticity
h2cr_ds['VORT'] = mpcalc.vorticity(h2cr_ds['U'], h2cr_ds['V'])

In [ ]:
h2pr_ds['VORT'].isel(time=12, lev=6).plot()

# hvPlot

In [ ]:
hvplot.help('quadmesh')

In [ ]:
def T_to_FLUT(T, unit='K'):
    if unit == 'C':
        T += 273.15
    sigma = 5.6693E-8
    olr = sigma*(T**4)
    
    return olr

In [ ]:
# https://cimss.ssec.wisc.edu/satellite-blog/wp-content/uploads/sites/5/2017/09/GOES16_CleanWindow_Landfall-20170920_0957_1136anim.gif


# Normalized cimss scale
bw_colors = [(0, '#BCBCBC'), (1, '#000000')]  
bw_cmp = LinearSegmentedColormap.from_list('FLUT bw', bw_colors, N=435)

levels = np.array([T_to_FLUT(temp, 'C') for temp in [-110, -105, -87.5, -80, -70, -60, -50, -35, -27.5, -22.5]])
fracs = levels-T_to_FLUT(-110, 'C')

rainbow_colors = [(0, '#0febff'), # cyan
                  ((fracs[1]/fracs[-1]), '#7f007f'), # purple
                  ((fracs[2]/fracs[-1]), '#e5e4e5'), # white
                  ((fracs[3]/fracs[-1]), '#000000'), # black
                  ((fracs[4]/fracs[-1]), '#ff0000'), # red
                  ((fracs[5]/fracs[-1]), '#FFFF00'), # yellow
                  ((fracs[6]/fracs[-1]), '#00FF00'), # green
                  ((fracs[7]/fracs[-1]), '#000073'), # navy
                  (1, '#00ffff')] # cyan


rainbow_cmp = LinearSegmentedColormap.from_list('FLUT colors', rainbow_colors, N=184)

bws = plt.get_cmap(bw_cmp)
bws_colors = bws(np.linspace(0, 1, 435))
rainbow = plt.get_cmap(rainbow_cmp)
r_colors = rainbow(np.linspace(0, 1, 184))

all_colors = np.vstack((r_colors, bws_colors))
flut_cimss = LinearSegmentedColormap.from_list('FLUT CIMSS', all_colors)
mpl.colormaps.register(flut_cimss, name='FLUT_CIMSS', force=True)

In [ ]:
plt.get_cmap(mpl.colormaps['FLUT_CIMSS'])

In [ ]:
?mpl.colormaps

In [ ]:
list(mpl.colormaps['FLUT CIMSS'])

In [ ]:
# FLUT CIMSS is my registered mpl colormap
project_coords=True

if project_coords == True:
    proj = ccrs.Robinson(central_longitude=180)
    g = h3cr_ds.FLUT.isel(time=slice(20, 30)).hvplot.quadmesh('lon', 'lat', project=proj, rasterize=True, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap='FLUT CIMSS', dynamic=False, frame_width=500)
else:
    g = h3cr_ds.FLUT.isel(time=slice(20, 30)).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap='FLUT CIMSS', dynamic=False, frame_width=500)
    
g

In [ ]:
h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, dynamic=False,
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap = 'FLUT_CIMSS',  coastline='10m', frame_width=500)

In [ ]:
h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap = 'viridis', dynamic=False, coastline='10m', frame_width=500)

In [ ]:
h3pr_ds.FLUT.isel(time=23).hvplot.quadmesh('lon', 'lat', geo=True, rasterize=True, clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')),
                             cmap = 'FLUT_CIMSS', dynamic=False, coastline='10m', frame_width=500)

In [ ]:
p_mesh_ds

In [ ]:
plats = p_mesh_ds.grid_center_lat
plons = p_mesh_ds.grid_center_lon

h3pn_ds.FLUT.isel(time=23).hvplot.quadmesh(plons, plats, geo=True, rasterize=True, dynamic=False, frame_width=500,
                                           clim=(T_to_FLUT(-110, 'C'), T_to_FLUT(55, 'C')), cmap='FLUT CIMSS', )

In [ ]:
help('hvplot.plot')

In [ ]:
hvplot.help('quadmesh')

# Plotly

In [ ]:
h1cr_ds

In [ ]:
h1cn_ds

In [ ]:
hvplot.extension('plotly')

# PyNGL

In [ ]:
# https://www.pyngl.ucar.edu/Examples/gallery.shtml
import Ngl, Nio

### MPAS

In [ ]:
# https://www.pyngl.ucar.edu/Examples/Scripts/mpas1.py
#---Read data from MPAS Grid
f  = Nio.open_file(c_mesh)
sp = f.variables["surface_pressure"][0,:]
sp = sp/1000.   # Not sure what the pressure units are, there's
                # not much metadata info on this file

lonCell = f.variables["lonCell"][:]
latCell = f.variables["latCell"][:]

#---Convert to degrees from radians
RAD2DEG   = 180.0/(math.atan(1)*4.0)  # Radian to Degree
lonCell   = lonCell * RAD2DEG
latCell   = latCell * RAD2DEG

#---Start the graphics
wks_type = "png"
wks = Ngl.open_wks(wks_type,"mpas1")

#---Read in desired color map so we can subset it later
cmap = Ngl.read_colormap_file("WhiteBlueGreenYellowRed")

res                      = Ngl.Resources()              # Plot mods desired.

res.cnFillOn             = True              # color plot desired
res.cnFillPalette        = cmap[48:208,:]    # Don't use white
res.cnLinesOn            = False             # turn off contour lines
res.cnLineLabelsOn       = False             # turn off contour labels
res.lbOrientation        = "Horizontal"      # vertical by default

res.trGridType           = "TriangularMesh"  # This is required to allow
                                             # missing coordinates.
res.cnLevelSelectionMode = "ManualLevels"
res.cnMinLevelValF       = 55
res.cnMaxLevelValF       = 100
res.cnLevelSpacingF      = 2.5
res.mpFillOn             = False
res.mpGridAndLimbOn      = False

res.sfXArray             = lonCell
res.sfYArray             = latCell

res.cnFillMode           = "RasterFill"      # turn raster on      
res.tiMainString         = "Surface pressure on MPAS grid ({} cells)".format(sp.shape[0])
res.tiMainFontHeightF   = 0.018

plot = Ngl.contour_map(wks,sp,res)  

Ngl.end()


### CAMSE

In [ ]:
# https://www.pyngl.ucar.edu/Examples/Scripts/camse1.py
import numpy, Nio, Ngl, sys, os

#---Read data
filename = "b.e12.B1850C5CN.ne30_g16.init.ch.027.cam.h0.0001-01.nc"
if(not os.path.exists(filename)):
  print("You do not have the necessary '{}' file to run this example.".format(filename))
  print("See the comments at the top of this script for more information.")
  sys.exit()

a     = Nio.open_file(filename)
vname = "TS"
data  = a.variables[vname]    
lat   = a.variables["lat"][:]        # 1D array (48602 cells)
lon   = a.variables["lon"][:]        # ditto

ncells = data.shape[1]
print("There are {} cells in the {} variable".format(ncells,vname))

wks_type = "png"
wks = Ngl.open_wks(wks_type,"camse1")

#---Set some plot options
res                   = Ngl.Resources()

# Contour options
res.cnFillOn          = True          # turn on contour fill
res.cnLinesOn         = False         # turn off contour lines
res.cnLineLabelsOn    = False         # turn off line labels
res.cnLevelSpacingF   = 2.5           # NCL chose 5.0
res.cnFillPalette     = "WhiteBlueGreenYellowRed"

# Map options
res.mpProjection      = "Orthographic"
res.mpCenterLonF      = 40
res.mpCenterLatF      = 60
res.mpPerimOn         = False

# Not sure why I need this.
res.pmTickMarkDisplayMode = "Never"

# Main Title
res.tiMainString      = f'{data.long_name} ({data.units}) ({ncells} cells)'
res.tiMainFontHeightF = 0.018

# Labelbar options
res.lbLabelFontHeightF = 0.01

#---Additional resources needed for putting contours on map
res.sfXArray          = lon
res.sfYArray          = lat

plot = Ngl.contour_map(wks,data[0,:],res)

Ngl.end()

# Paul's NCL Code

In [ ]:
# https://github.com/SEATStandards/ncvis



# psyplot

In [ ]:
# https://psyplot.github.io/examples/maps/example_ugrid.html